# Neo4J - with Python #
##### Lukas Hinterleitner, Daniel Nepp #####
For a complete documentation visit: https://neo4j.com/developer/python/

---

## Installation ##
### 1. Download and install Neo4j Desktop: ###
https://neo4j.com/download/

Before you can download the installation file you have to provide your data.
If you don't want give your data away, you can type in random stuff :)

An activation key gets displayed while downloading (shown in the picture below).
Keep it, you will need it later during the installation.
<br><br>

![Neo4J Activation Key](screenshots/key.png)
<br><br>

After the installation finished you should see the home screen which is displayed in the image below.
We will get back to it later when we create our first Neo4J graph database.

![Home screen Neo4J Desktop](screenshots/home_screen.png)
<br><br>

### 2. Create a Conda-Environment : ###

`$ conda create --name neo4j python=3.8.2`


### 3. Activate Conda-Environment : ###

`$ conda activate neo4j`


### 4. Install the Neo4j driver : ###
https://anaconda.org/conda-forge/neo4j-python-driver

`$ conda install -c conda-forge neo4j-python-driver`

### 5. Install jupyter notebook : ###

`$ conda install -c anaconda jupyter`

or install jupyter lab

`$ conda install -c conda-forge jupyterlab`

or install jupyter in pycharm...


## Creating Neo4J Database : ##

Description...

![Home screen Neo4J Desktop](screenshots/create_local_database.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/create_database.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/database_after_installation.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/database_settings.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/neo4j_browser_home_screen.png)
<br><br>

---

## Using Neo4j : ##

The code below is a Neo4J Python hello world example! :)


In [2]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    greeting = session.write_transaction(_create_and_return_greeting, "Hello World!")
    print(greeting)

driver.close()

Hello World!, from node 3


### 1. Create Neo4J Entities : ###

```sql
CREATE (
        TheMatrix:Movie
        {title:'The Matrix', released:1999, tagline:'Welcome to the Real World'}
        )
```

```sql
CREATE (
        Keanu:Person
        {name:'Keanu Reeves', born:1964}
        )
```

### 2. Create Neo4J Relations : ###

```sql
(Keanu) - [:ACTED_IN { roles : [ 'Neo' ]}] -> (TheMatrix)
```

```sql
(LillyW) - [:DIRECTED] -> (TheMatrix)
```

### 3. Find Neo4J Entities : ###

Find Keanu Reeves - returns single entity:
```sql
MATCH (keanu {name: "Keanu Reeves"}) RETURN keanu
```

Find The Matrix - returns single entity:
```sql
MATCH (theMatrix {title: "The Matrix"}) RETURN theMatrix
```

Find ten people - returns ten entities:
```sql
MATCH (people:Person) RETURN people.name LIMIT 10
```

Find movies produced in the 90's - returns all movies between 1990 and 2000:
```sql
MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN nineties.title
```

List all Keanu Reeves movies - returns all movies where Keanu Reeves acted in:
```sql
MATCH (keanu:Person {name: "Keanu Reeves"}) - [:ACTED_IN] -> (keanuMovies) RETURN keanu,keanuMovies
```

Who directed "The Matrix"? - returns all directors of The Matrix:
```sql
MATCH (theMatrix {title: "The Matrix"}) <- [:DIRECTED] - (directors) RETURN directors.name
```

Keanu Reeves' co-actors -- returns all actors who acted in any movie aside Keanu Reeves:
```sql
MATCH (keanu:Person {name:"Keanu Reeves"}) - [:ACTED_IN] -> (m) <- [:ACTED_IN] - (coActors) RETURN coActors.name
```

How people are related to "The Matrix" - returns all people inclusive their relations to the movie The Matrix:
```sql
MATCH (people:Person) - [relatedTo] - (:Movie {title: "The Matrix"}) RETURN people.name, Type(relatedTo), relatedTo
```

Movies and actors up to 4 "hops" away from Kevin Bacon:
```sql
MATCH (bacon:Person {name:"Kevin Bacon"}) - [*1..4] - (hollywood)
RETURN DISTINCT hollywood
```
Note: hollywood here is just a placeholder for any entity

Bacon path, the shortest path of any relationships to Meg Ryan:
```sql
MATCH p=shortestPath(
(bacon:Person {name:"Kevin Bacon"}) - [*] - (meg:Person {name:"Meg Ryan"})
)
RETURN p
```

Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
    (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND tom <> cocoActors
RETURN cocoActors.name AS Recommended, count(*) AS Strength ORDER BY Strength DESC
```

Find someone to introduce Tom Hanks to Tom Cruise:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
  (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

### 4. Delete Neo4J Graph : ###
Delete all Movie and Person nodes, and their relationships:
```sql
MATCH (n) DETACH DELETE n
```

Note you only need to compare property values like this when first creating relationships
Prove that the Movie Graph is gone
```sql
MATCH (n) RETURN n
```


## Let's have some fun : ##

In [3]:
# importing GraphDatabase
from neo4j import GraphDatabase

# Connection data :
uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

# Connecting...
driver = GraphDatabase.driver(uri, auth=(user, password))


... now let's run queries ...

In [60]:
# resetting database

with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    _q("MATCH (n) DETACH DELETE n") # remove all graphs and nodes

    #---------------------------------------
driver.close()

... insert a person ...

In [62]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    persons = [
        {'name':'Jammie Tullin', 'born':'1999'},
        {'name':'Tina Tuna', 'born':'1995'},
        {'name':'Marry Murry', 'born':'1992'},
        {'name':'Julian Jingle', 'born':'1965'},
        {'name':'Sam Sum', 'born':'1987'},
        {'name':'Lukas Hinterleitner', 'born':'1998'},
        {'name':'Daniel Nepp', 'born':'1997'}
    ] 

    for person in persons:
        result = _q("CREATE (p:Person {name:'%s', born:'%s'}) RETURN p" % (person['name'],person['born']))
        for record in result: print("Person created:", record['p']['name']) 

    #---------------------------------------
driver.close()

person created: Jammie Tullin
person created: Tina Tuna
person created: Marry Murry
person created: Julian Jingle
person created: Sam Sum
person created: Lukas Hinterleitner
person created: Daniel Nepp


In [63]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    movies = [
        {'title':'Neo4j', 'year':'2020'},
        {'title':'Matrix4j', 'year':'1234'},
        {'title':'Titanic4j', 'year':'2008'},
        {'title':'4j4j', 'year':'2023'},
        {'title':'Python - attack of the snake', 'year':'2004'}, 
        {'title':'Mr.Bean4j', 'year':'2006'},
    ] 

    for movie in movies:
        result = _q("CREATE (m:Movie {title:'%s', year:'%s'}) RETURN m" % (movie['title'],movie['year']))
        for record in result: print("Movie created:", record['m']['title']) 

    #---------------------------------------
driver.close()

Movie created: Neo4j
Movie created: Matrix4j
Movie created: Titanic4j
Movie created: 4j4j
Movie created: Python - attack of the snake
Movie created: Mr.Bean4j


In [73]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    relations = [
        { 'name':'Daniel Nepp', 'type':'STARRED_IN', 'title':'Neo4j' }, 
        { 'name':'Lukas Hinterleitner', 'type':'STARRED_IN', 'title':'Neo4j' }, 
        { 'name':'Daniel Nepp', 'type':'DIRECTED', 'title':'Neo4j' }, 
        { 'name':'Lukas Hinterleitner', 'type':'DIRECTED', 'title':'Neo4j' }, 
        { 'name':'Daniel Nepp', 'type':'DIRECTED', 'title':'Mr.Bean4j' }, 
        { 'name':'Lukas Hinterleitner', 'type':'DIRECTED', 'title':'Titanic4j' }, 
        
        { 'name':'Jammie Tullin', 'type':'STARRED_IN', 'title':'Python - attack of the snake' }, 
        { 'name':'Tina Tuna',  'type':'STARRED_IN', 'title':'Python - attack of the snake' },
        { 'name':'Jammie Tullin', 'type':'STARRED_IN', 'title':'4j4j' }, 
        { 'name':'Tina Tuna',  'type':'STARRED_IN', 'title':'4j4j' },
        { 'name':'Marry Murry', 'type':'STARRED_IN', 'title':'Mr.Bean4j' },
        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'title':'Mr.Bean4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'title':'Mr.Bean4j' },
        
        { 'name':'Marry Murry', 'type':'STARRED_IN', 'title':'Neo4j' },
         
        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'title':'Matrix4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'title':'Matrix4j' },
        
        { 'name':'Julian Jingle', 'type':'STARRED_IN', 'title':'Titanic4j' },
        { 'name':'Sam Sum', 'type':'STARRED_IN', 'title':'Titanic4j' }
        
        
    ] 

    for relation in relations: 
        result = _q(
            """
                MATCH (p:Person),(m:Movie)
                WHERE p.name = '{0}' AND m.title = '{2}'
                CREATE (p)-[r:{1}]->(m)
                RETURN p.name, type(r), m.title
            """
            .format( relation['name'], relation['type'], relation['title'] )
        )
        for record in result: print("Relation created:", record['p.name'], record['type(r)'], record['m.title']) 

    #---------------------------------------
driver.close()

Relation created: Daniel Nepp STARRED_IN Neo4j
Relation created: Lukas Hinterleitner STARRED_IN Neo4j
Relation created: Daniel Nepp DIRECTED Neo4j
Relation created: Lukas Hinterleitner DIRECTED Neo4j
Relation created: Daniel Nepp DIRECTED Mr.Bean4j
Relation created: Lukas Hinterleitner DIRECTED Titanic4j
Relation created: Jammie Tullin STARRED_IN Python - attack of the snake
Relation created: Tina Tuna STARRED_IN Python - attack of the snake
Relation created: Jammie Tullin STARRED_IN 4j4j
Relation created: Tina Tuna STARRED_IN 4j4j
Relation created: Marry Murry STARRED_IN Mr.Bean4j
Relation created: Julian Jingle STARRED_IN Mr.Bean4j
Relation created: Sam Sum STARRED_IN Mr.Bean4j
Relation created: Marry Murry STARRED_IN Neo4j
Relation created: Julian Jingle STARRED_IN Matrix4j
Relation created: Sam Sum STARRED_IN Matrix4j
Relation created: Julian Jingle STARRED_IN Titanic4j
Relation created: Sam Sum STARRED_IN Titanic4j


In [83]:
with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------
 
    result = _q(
        """
            MATCH 
                (p:Person {name:"Daniel Nepp"} ) - [:STARRED_IN] -> (m) <- [:STARRED_IN] - (coStar) 
            WHERE coStar.name <> p.name
            RETURN DISTINCT coStar.name
        """
    )
    for record in result: print("Daniel's co-star found:", record['coStar.name'] ) 

    #---------------------------------------
driver.close()

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown operation '!=' (you probably meant to use '<>', which is the operator for inequality testing) (line 4, column 31 (offset: 151))
"            WHERE coStar.name != p.name"
                               ^}